## 1. Clip the DEM to the catchment shape file and cut at Napolean III Bridge

Convert the coordinates of the bridge from WGS84 to Lambert93

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import rasterio as rio

from osgeo import gdal
from pyproj import Proj, Transformer

## Landlab components
from landlab.components import SinkFiller # SinkFiller is optional
from landlab.io import read_esri_ascii, write_esri_ascii
from landlab.plot import imshow_grid  # plotter functions are optional

In [ ]:
transformer = Transformer.from_crs("epsg:4326", "epsg:2154")

lat = 43.66369
lon = 7.19818
x, y = transformer.transform(lat, lon)
print('{}, {}'.format(x, y))

Separate Qgis installation within its own environment, to stop caesarpy becoming too bloated:

```
conda activate qgis_stable
python crop-dem-to-catchment.py
```

## 2. Resample DEM at at lower resultion using mean and minimum critera

Again, using Qgis, so run the following python script
```
python resample-dem.py
```
and now there are two DEMs

In [ ]:
path_to_tif = "./remaud/"
resampled_raster = '{}dem_200m_mean.tif'.format(path_to_tif)

dem = rio.open(resampled_raster)  # read in tif file
topo_mean = dem.read(1).astype(float)  # assume that elevation is in the first band
topo_mean[topo_mean <= 0] = np.nan  # replace -9999 with nan

resampled_raster = '{}dem_200m_min.tif'.format(path_to_tif)

dem = rio.open(resampled_raster)  # read in tif file
topo_min = dem.read(1).astype(float)  # assume that elevation is in the first band
topo_min[topo_min <= 0] = np.nan  # replace -9999 with nan

# Gqis has left a buffer around the DEM, so the bottom does not touch the boundary.
# Remove bottom two rows
topo_mean = topo_mean[:-1,:]
topo_min = topo_min[:-1,:]

plt.figure(figsize=[20,10])
plt.subplot(131)
plt.imshow(topo_mean)
plt.title('Mean')
plt.subplot(132)
plt.imshow(topo_min)
plt.title('Min')
plt.subplot(133)
plt.imshow(topo_mean-topo_min)
plt.title('Difference')
plt.show()

## 3. Run D4 pit filling algorithm on minimum DEM

I will convert the DEM to ascii format and then run the `SinkFiller` algorithm from `LandLab` on the minimim 200m DEM.

In [ ]:
def numpy2asc(array,tiffile, ascfile):
    """
    Function to convert a geotiff (.tif) file into an ascii file (.asc)
    :param tiffile: input geotiff file name
    :param ascfile: output ascii file name (CAESAR expects .asc)
    :return: None
    """
    rio_array = rio.open(tiffile)  # read in tif file
    
    f = open(ascfile, 'w')
    f.write('ncols         {}\n'.format(np.shape(array)[1]))
    f.write('nrows         {}\n'.format(np.shape(array)[0]))
    f.write('xllcorner     {}\n'.format((rio_array.transform * (0, rio_array.height))[0]))
    f.write('yllcorner     {}\n'.format((rio_array.transform * (0, rio_array.height))[1]))
    f.write('cellsize      {}\n'.format(rio_array.transform[0]))
    f.write('NODATA_value  -9999\n')

    pix = '{0} '
    for i in range(np.shape(array)[0]):
        for j in range(np.shape(array)[1]):
            if np.isnan(array[i, j]) == True:
                f.write(pix.format(int(-9999)))  # need to replace nan with -9999
            elif array[i, j] <= 0:
                f.write(pix.format(int(-9999)))  # need to replace 0 with -9999
            else:
                f.write(pix.format(array[i, j]))
        f.write("\n")
    f.close()

In [ ]:
dem_asc = '{}dem_200m_min.asc'.format(path_to_tif)
numpy2asc(topo_min, resampled_raster, dem_asc)

In [ ]:
# Load in minimum DEM
(rmg, z) = read_esri_ascii(dem_asc, name='topographic__elevation')

# run the sink filling algorithm
rmg.set_watershed_boundary_condition(z)
sf = SinkFiller(rmg, routing='D4', apply_slope=False)
sf.fill_pits()

dem_filled = '{}dem_200m_min_filled.asc'.format(path_to_tif)
if os.path.isfile(dem_filled):
    os.remove(dem_filled)
write_esri_ascii(dem_filled, rmg, names='topographic__elevation')

In [ ]:
def asc2numpy(ascfile):
    """
    Function to read in an ascii file (.asc) and return a Numpy array
    :param ascfile: input ascii file name
    :return ncols: number of columns
    :return nrows: number of rows
    :return geotransform: the geo-transfrom info
    :return data: a numpy array of the ascii data
    """

    try:
        gdal_data = gdal.Open(ascfile)  # use GDAL to open the ascii fle
        ncols = gdal_data.RasterXSize  # get the number of columns
        nrows = gdal_data.RasterYSize  # get the number of rows
        geotransform = gdal_data.GetGeoTransform()  # get the geo-transform info
        data_array = gdal_data.ReadAsArray().astype(float)  # convert the data into a numpy array
        data_array[data_array <= -9999] = np.nan  # replace -9999 with nan

        return ncols, nrows, geotransform, data_array

    except:
        print('cannot open {}'.format(ascfile))


In [ ]:
nx, ny, geotransform, dem_min = asc2numpy(dem_asc)
nx, ny, geotransform, dem_fill = asc2numpy(dem_filled)

plt.figure(figsize=[20,10])
plt.subplot(121)
plt.imshow(dem_fill)
plt.colorbar()
plt.subplot(122)
plt.imshow(dem_fill - dem_min)
plt.colorbar()
plt.title('Filled Pits')
plt.show()

## 4. Run CL on the pit filled minimum DEM to get flow network

This step is to allow us to sample the regions of the pit filled minimum DEM to use to replace the mean resampled DEM in the final step.
```
cp remaud/dem_200m_min_filled.asc remaud/hydrology-only-model/input_data/.
cd remaud/hydrology-only-model
bash run_HC.sh
```


Take the final output from the hydrology only run to see where water collects within the DEM

In [ ]:
dem_water_file = 'remaud/hydrology-only-model/results/WaterDepths216000.asc'
nx, ny, geotransform, dem_water = asc2numpy(dem_water_file)

x = np.linspace(0, nx-1, nx)
y = np.linspace(ny-1, 0, ny)
X,Y = np.meshgrid(x,y, indexing='xy')

plt.figure(figsize=[10,10])
plt.pcolormesh(X, Y, dem_water, norm=colors.LogNorm(vmin=1e-3, vmax=1e-1), shading='auto')
plt.colorbar()
plt.title('Water Depth')
plt.show()

## 5 Replace regions with flow pit filled minimum DEM

I chose to replace values from the mean 200 m DEM with those from the pit filled minimum DEM when the water depth is greater than 0.05 m

In [ ]:
topo_preproc = topo_mean.copy()
topo_preproc[dem_water > 0.05] = dem_fill[dem_water > 0.05]

In [ ]:
dem_asc = '{}dem_200m_preproc.asc'.format(path_to_tif)
numpy2asc(topo_preproc, resampled_raster, dem_asc)

In [ ]:
plt.figure(figsize=[17,5])
plt.subplot(131)
plt.pcolormesh(X, Y, topo_preproc)
plt.title('Preprocessed')
plt.colorbar()
plt.subplot(132)
plt.pcolormesh(X, Y, topo_mean)
plt.colorbar()
plt.title('Mean')
plt.subplot(133)
plt.pcolormesh(X, Y, topo_mean - topo_preproc, norm=colors.LogNorm(vmin=1e-1, vmax=1e2), cmap='magma_r', shading='auto')
plt.colorbar()
plt.title('Difference')
plt.show()